In [23]:
using DataFrames, CSV
using JLD
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays
using Plots, StatsPlots
using Distributions

In [24]:
liverpool_data = CSV.read("Liverpool_FIFA_22.csv", DataFrame)
delete!(liverpool_data, [19, 21, 23, 28, 29, 30, 31, 32, 33])
n_players = nrow(liverpool_data)

24

In [25]:
all_teams = CSV.read("Team_Ratings.csv", DataFrame)
n_positions = 27
C = CSV.read("fixture_list.csv", DataFrame)
n_games = nrow(C)

38

In [26]:
GK = "gk"
defenders = ["lwb", "rwb", "lb", "lcb", "cb", "rcb", "rb"]
midfielders = ["lam", "cam", "ram", "lm", "lcm", "cm", "rcm", "rm", "ldm", "cdm", "rdm"]
forwards = ["ls", "st", "rs", "lw", "lf", "cf", "rf", "rw"]
center_backs = ["lcb", "cb", "rcb"]
right_backs = ["rwb", "rb"]
left_backs = ["lwb", "lb"]
cdms = ["ldm", "cdm", "rdm"]
cms = ["lcm", "cm", "rcm"]
cams = ["lam", "cam", "ram"]
left_wingers = ["lw", "lm"]
right_wingers = ["rw", "rm"]
strikers = ["ls", "st", "rs", "lf", "cf", "rf"]

6-element Vector{String}:
 "ls"
 "st"
 "rs"
 "lf"
 "cf"
 "rf"

In [27]:
player_pos = ["ls", "st", "rs", "lw", "lf", "cf", "rf", "rw", "lam", "cam", "ram", "lm", "lcm", "cm", "rcm", "rm", "lwb", "ldm", "cdm", "rdm", "rwb", "lb", "lcb", "cb", "rcb", "rb", "gk"]
pos_to_ind = Dict()
ind_to_pos = Dict()
index = 1
for pos in player_pos
    pos_to_ind[pos] = index
    ind_to_pos[index] = pos
    index += 1
end
pos_to_ind["st"]

2

In [28]:
defensive_pos = Set(["lb", "lcb", "cb", "rcb", "rb", "ldm", "cdm", "rdm"])
central_pos = Set(["ldm", "cdm", "rdm", "lcm", "cm", "rcm", "lam", "cam", "ram"])
forward_pos = Set(forwards)
wing_pos = Set(["lwb", "rwb", "lm", "rm", "lw", "rw"])
wingback_pos = Set(["lwb", "rwb", "lb", "rb"])
union(wingback_pos, wing_pos)

Set{String} with 8 elements:
  "rm"
  "rwb"
  "lb"
  "rw"
  "lw"
  "lm"
  "lwb"
  "rb"

In [29]:
function acceptable_positions(player_orig)
    acceptable_pos = Set()
    for k in player_orig
        push!(acceptable_pos, lowercase(k))
    end
    return acceptable_pos
end

R = zeros(Int64, (n_players, n_positions))
for i in 1:n_players
    player_orig = split(liverpool_data[i, "player_positions"], ", ")
    acceptable_pos = acceptable_positions(player_orig)
    for pos in acceptable_pos
        R[i, pos_to_ind[pos]] = liverpool_data[i, pos]
    end
end
R[1, :]

27-element Vector{Int64}:
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  ⋮
  0
  0
  0
  0
  0
  0
  0
  0
 89
  0
  0
  0

In [30]:
pos_discount = 0.99
function penalty(player_number, game_number, x)
    if game_number == 1
        return 1
    end
    if sum(x[player_number, game_number-1, k] for k in 1:n_positions) == 0
        return 1
    end
    max_consecutive = 0
    for j in game_number-1:-1:1
        if sum(x[player_number, j, k] for k in 1:n_positions) == 0
            break
        else
            max_consecutive += 1
        end
    end
    return pos_discount^max_consecutive
end

penalty (generic function with 1 method)

In [31]:
# model = Model(with_optimizer(Gurobi.Optimizer))
model = Model(Gurobi.Optimizer)

### Decision Variables ###

### x[i][j][k]. player i, game j, position k ###
@variable(model, x[i=1:n_players, j=1:n_games, k=1:n_positions], Bin)

### z variable, for whether we go for win or not ###
@variable(model, z[j = 1:n_games], Bin)


### Match Winning Constraint ###
### R is player ratings, R[i][k] is player i for position k ###
### C is the opposing team rating, C[j] is the rating of the opposing team for game j ###

for j in 1:n_games
    @constraint(model, sum(sum(R[i, k]*x[i, j, k] for i in 1:n_players) for k in 1:n_positions)/11 >= C[j, 2]*z[j])
end

### Player Constraints ###

### Each player can only play at most once in a game ###
for j in 1:n_games
    for i in 1:n_players
        @constraint(model, sum(x[i, j, k] for k in 1:n_positions) <= 1)
    end
end

### Can only have 11 players in game ###
for j in 1:n_games
    @constraint(model, sum(sum(x[i, j, k] for k in 1:n_positions) for i in 1:n_players) == 11)

    ### 1 GK per game ###
    @constraint(model, sum(x[i, j, pos_to_ind[GK]] for i in 1:n_players) == 1)

    ### 4 Defenders per game ###
    @constraint(model, sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in defenders) == 4)

    ### 3 - 5 Midfielders per game ### 
    @constraint(model, 3 <= sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in midfielders) <= 5)

    ### 1 - 3 Forwards per game ###
    @constraint(model, 1 <= sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in forwards) <= 3)

    # ### 2 CBs per game ###
    @constraint(model, sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in center_backs) == 2)

    # ### 1 RB per game ###
    @constraint(model, sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in right_backs) == 1)

    # ### 1 LB per game ###
    @constraint(model, sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in left_backs) == 1)

    # ### 0 - 2 CDMs per game ###
    @constraint(model, 0 <= sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in cdms) <= 2)

    # ### 0 - 3 CMs per game ###
    @constraint(model, 0 <= sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in cms) <= 3)

    # ### 0 - 3 CAMS per game ###
    @constraint(model, 0 <= sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in cams) <= 3)

    # ### 0 - 1 LW, LM per game ###
    @constraint(model, sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in left_wingers) <= 1)

    # ### 0 - 1 RW, RM per game ###
    @constraint(model, sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in right_wingers) <= 1)

    # ### 1 ST per game ###
    @constraint(model, sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in strikers) == 1)
end

### Objective Function ###
@objective(model, Max, sum(z[j] for j in 1:n_games))

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-03


z[1] + z[2] + z[3] + z[4] + z[5] + z[6] + z[7] + z[8] + z[9] + z[10] + z[11] + z[12] + z[13] + z[14] + z[15] + z[16] + z[17] + z[18] + z[19] + z[20] + z[21] + z[22] + z[23] + z[24] + z[25] + z[26] + z[27] + z[28] + z[29] + z[30] + z[31] + z[32] + z[33] + z[34] + z[35] + z[36] + z[37] + z[38]

In [32]:
optimize!(model)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1482 rows, 24852 columns and 99142 nonzeros
Model fingerprint: 0x532f7cd1
Variable types: 190 continuous, 24662 integer (24662 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 1445 rows and 24562 columns
Presolve time: 0.21s
Presolved: 37 rows, 290 columns, 781 nonzeros
Variable types: 0 continuous, 290 integer (289 binary)

Root relaxation: objective 3.800000e+01, 20 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   38.00000    0    2          -   38.00000      -     -    0s
H    0     0                      37.0000000   38.00000  2.70% 

In [33]:
opt_x = value.(x)
function matrix_to_lineup(x, j)
    player_to_pos = Dict()
    for i in 1:n_players
        for k in 1:n_positions
            cur_val = x[i, j, k]
            if cur_val == 1
                player_name = liverpool_data[i, "short_name"]
                player_pos = ind_to_pos[k]
                player_to_pos[player_name] = player_pos
            end
        end
    end
    return player_to_pos
end

println(matrix_to_lineup(opt_x, 1))

Dict{Any, Any}(String31("S. Mané") => "lw", String31("J. Matip") => "cb", String31("J. Milner") => "cm", String31("A. Oxlade-Chamberlain") => "cm", String31("H. Elliott") => "rw", String31("V. van Dijk") => "ram", String31("D. Origi") => "st", String31("N. Williams") => "rb", String31("A. Robertson") => "lb", String31("Alisson") => "gk", String31("I. Konaté") => "cb")


In [34]:
function every_lineup(x)
    game_to_lineup = Dict()
    for j in 1:n_games
        game_to_lineup[j] = matrix_to_lineup(x, j)
    end
    return game_to_lineup
end

line_ups = every_lineup(opt_x)

Dict{Any, Any} with 38 entries:
  5  => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Matip")=>"cb", S…
  16 => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Matip")=>"cb", S…
  20 => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Matip")=>"cb", S…
  35 => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Matip")=>"cb", S…
  12 => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Matip")=>"cb", S…
  24 => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Matip")=>"cb", S…
  28 => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Matip")=>"cb", S…
  8  => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Matip")=>"cb", S…
  17 => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Matip")=>"cb", S…
  30 => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Matip")=>"cb", S…
  1  => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Matip")=>"cb", S…
  19 => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Matip")=>"cb",

In [35]:
line_ups[1]

Dict{Any, Any} with 11 entries:
  String31("S. Mané")               => "lw"
  String31("J. Matip")              => "cb"
  String31("J. Milner")             => "cm"
  String31("A. Oxlade-Chamberlain") => "cm"
  String31("H. Elliott")            => "rw"
  String31("V. van Dijk")           => "ram"
  String31("D. Origi")              => "st"
  String31("N. Williams")           => "rb"
  String31("A. Robertson")          => "lb"
  String31("Alisson")               => "gk"
  String31("I. Konaté")             => "cb"